In [1]:
cd ..

d:\law_graph_agent


In [2]:
import random
import numpy as np
import pandas as pd
from datasets import load_dataset
from modules.eval.retrieval.hybrid_engine import HybridEvaluationEngine

c:\ProgramData\miniconda3\envs\law_agent\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


resource module not available on Windows


In [3]:
# Configuration
bm25_weight = 0.5
dense_weight = 0.5
rrf_k = 60.0
batch_size = 32
dimension = 768

models = {
    "bkai": {
        "model_name": "bkai-foundation-models/vietnamese-bi-encoder",
        "word_segment": True
    },
    "halong": {
        "model_name": "hiieu/halong_embedding",
        "word_segment": False
    }
}

dataset_name_base = "another-symato/VMTEB-Zalo-legel-retrieval"
num_corpus_addition = 0

eval_results = {}

for model_key, model_config in models.items():
    dataset_name = dataset_name_base + "-wseg" if model_config["word_segment"] else dataset_name_base
    
    # Load datasets
    corpus = load_dataset(dataset_name, "corpus", split='train')
    queries = load_dataset(dataset_name, "queries", split='train')
    relevant_docs_data = load_dataset(dataset_name, "data_ir", split="test")
    
    # Filter corpus and queries based on relevant docs
    required_corpus_ids = list(map(str, relevant_docs_data["corpus_id"]))
    if num_corpus_addition:
        required_corpus_ids += random.sample(corpus["id"], k=num_corpus_addition)
    corpus = corpus.filter(lambda x: x["id"] in required_corpus_ids)
    
    required_queries_ids = list(map(str, relevant_docs_data["query_id"]))
    queries = queries.filter(lambda x: x["query_id"] in required_queries_ids)
    
    # Convert to dictionary format
    corpus_dict = dict(zip(corpus["id"], corpus["text"]))
    queries_dict = dict(zip(queries["query_id"], queries["question"]))
    relevant_docs = {}
    for qid, corpus_id in zip(relevant_docs_data["query_id"], relevant_docs_data["corpus_id"]):
        qid = str(qid)
        corpus_id = str(corpus_id)
        if qid not in relevant_docs:
            relevant_docs[qid] = set()
        relevant_docs[qid].add(corpus_id)
    
    # Initialize evaluation engine
    engine = HybridEvaluationEngine(
        model_name=model_config["model_name"],
        bm25_weight=bm25_weight,
        dense_weight=dense_weight,
        rrf_k=rrf_k,
        dimension=dimension
    )
    
    # Run evaluation
    eval_results[model_key] = engine.evaluate(queries, corpus, relevant_docs)

# Create score board
df_score_board = pd.DataFrame(eval_results)

c:\ProgramData\miniconda3\envs\law_agent\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'modules.eval.base_engine'

In [27]:
df_score_board.to_json('output/scores.json')